## **1. Regroupement en sous-corpus**
Plutôt que de récupérer un tag qui aurait été assigné manuellement pour chaque lien, on va le créer en déconstruisant les URLs et en créant des sous-dossiers correspondant à différents sous-niveaux dans l'arborescence du site.  

À décider : jusqu'à quel niveau on regroupe (ça peut être un peu du cas par cas selon l'organisation du site)

Créer des sous-dossiers à l'intérieur

In [190]:
path = '/Users/camilledemers/Documents/03-corpus/0-listes_sites_html_pdf/'
acteur = "chu_iu"
sous_acteur = 'chu_qc'

from pandas import *

# encoding= 'ISO-8859-1' "utf-8"
with open(path + acteur  + '.csv', encoding='ISO-8859-1') as f:
    csv = read_csv(f, sep=';')

In [191]:
csv

,Address,Type,Title,Charset,Description
0,https://www.chudequebec.ca/,text/html,<h1>CHU de Québec...,utf-8,NaN
1,http://iucpq.qc.ca/,text/html,NaN,NaN,NaN
2,http://www.chumontreal.qc.ca/,text/html,"<img alt=""CHUM"" src=""/getattachment/a-propos/L...",iso-8859-1,NaN
3,https://cusm.ca/,text/html,Centre universitaire de santé McGill,utf-8,The McGill University Health Centre (MUHC) is ...
4,https://www.chusj.org/,text/html,CHU Sainte-Justine : h&#244;pital m&#232;re en...,utf-8,Le Centre hospitalier universitaire Sainte-Jus...
...,...,...,...,...,...
9668,https://www.chudequebec.ca/a-propos-de-nous/pu...,text/html,redir,utf-8,NaN
9669,https://www.chudequebec.ca/a-propos-de-nous/pu...,text/html,2020,utf-8,NaN
9670,https://www.chudequebec.ca/a-propos-de-nous/pu...,text/html,2021,utf-8,NaN
9671,https://www.chudequebec.ca/getattachment/22688...,text/html,NaN,utf-8,NaN


In [192]:
liste = [lien for lien in csv['Address'].tolist() if not 'actualites' in lien and not 'propos' in lien and not 'joindre' in lien and not 'communiques' in lien and not 'about' in lien]
liste = [lien for lien in liste if 'chudequebec.ca' in lien]

In [193]:
liste

['https://www.chudequebec.ca/',
 'https://www.chudequebec.ca/accueil.aspx',
 'https://www.chudequebec.ca/ScriptResource.axd?d=4SGRTDi6q2Unw0sZeAdCnhkWxDjdKzfZCRlqXtF4KLqRRiweWisBVa5-L4CGzRdV3rhPeB4l7BPiti0GJO3Q38FCFHpESH9zIk5p2SkVs8SphQXhzZeRAedHZ6znZBA9xb20D_FsWNyFdwAtSYNv96TWaFw-oeVxcqBjI1HxIc01&t=ffffffffaa493ab8',
 'https://www.chudequebec.ca/patient/maladies,-soins-et-services/m-informer-sur-les-soins-et-services/centre-integre-de-cancerologie-(cic).aspx',
 'https://www.chudequebec.ca/visiteur/connaitre-les-heures-de-visites.aspx',
 'https://www.chudequebec.ca/professionnels-de-la-sante/antibiogouvernance.aspx',
 'https://www.chudequebec.ca/professionnels-de-la-sante/cancerologie.aspx',
 'https://www.chudequebec.ca/professionnels-de-la-sante/centre-des-maladies-du-sein-(cms)/centre-des-maladies-du-sein-(cms).aspx',
 'https://www.chudequebec.ca/professionnels-de-la-sante/centre-des-maladies-du-sein-(cms)/centre-des-maladies-du-sein-(cms)/outils-pour-les-pharmaciens.aspx',
 'https:/

In [238]:
"""
0 = http/s
1 = //
2 = domaine principal (ex. INSPQ)
3 = 1e sous-niveau
4 = 2e sous-niveau
5 = 3e sous-niveau
etc.
"""

liens_2N = [lien for lien in liste if len(lien.split('/')) >= 3]
liens_3N = [lien for lien in liste if len(lien.split('/')) >= 4]
liens_4N = [lien for lien in liste if len(lien.split('/')) >= 5]

In [239]:
lien = liste[20]
lien.split('/')

['https:',
 '',
 'www.chudequebec.ca',
 'professionnels-de-la-sante',
 'clinique-interdisciplinaire-de-memoire-(cime)',
 'chaires-de-recherche.aspx']

In [240]:
regroupements = {}

for lien in liens_2N:
    tag = lien.split('/')[3]
    if tag in regroupements:
        regroupements[tag].append(lien)
        
    else:
        if len(tag)>1:
            regroupements[tag] = []

tags = list(regroupements.keys())

In [241]:
# On garde seulement les sections qui contiennent plus de 15 sous-pages
tags_retenus = [tag for tag in tags if len(regroupements[tag]) >10]

In [242]:
print('Ça nous donne {} tags.'.format(len(tags_retenus)))
tags_retenus

Ça nous donne 10 tags.


['patient',
 'visiteur',
 'professionnels-de-la-sante',
 'getmedia',
 'getattachment',
 'centre-hospitaliers',
 'emplois',
 'dons-et-benevolat',
 'chudequebec.ca',
 'Actualites']

In [160]:
import os, shutil
from pathlib import Path

# Créer un sous-dossier où on va aller copier les fichiers .txt correspondant au sous-corpus
path='/Users/camilledemers/Documents/03-corpus/2-sous-corpus/' + acteur + '/'

Path(path).mkdir(parents=True, exist_ok=True)


for tag in tags_retenus:
    subfolder =  tag + '/' #le nom du tag

    Path(path+subfolder).mkdir(parents=True, exist_ok=True)

    # Aller chercher les fichiers .txt correspondant au sous-corpus dans le corpus principal et les copier dans le sous-dossier qu'on vient de créer
    src = '/Users/camilledemers/Documents/03-corpus/2-data/1-fr/' + acteur + '/' 
    os.chdir(src)
    dst = path + subfolder

    for file in os.listdir():
        if (file.endswith(".txt") and not "corpus" in file):
            file_path = src + file
            
            with open(file_path, 'r', encoding = "UTF-8") as f:
                data = f.readlines()
                try: 
                    url = data[0]
                    if(tag in url):
                        shutil.copy(src + file, dst)
                except:
                    True
                    

In [161]:
path='/Users/camilledemers/Documents/03-corpus/2-sous-corpus/' + acteur + '/'
liste_total = sum([len(files) for r, d, files in os.walk(path)])

In [162]:
path='/Users/camilledemers/Documents/03-corpus/2-sous-corpus/' + acteur + '/'
os.chdir(path)

liste_fichiers = []
for subfolder in os.listdir():
    os.chdir(path + subfolder)
    liste_fichiers += os.listdir()

    os.chdir(path)

liste_fichiers = set(liste_fichiers)

In [163]:
os.chdir(src)
len_corpus_global = len(os.listdir())
print('Pages dans le corpus global : ' + str(len_corpus_global))

os.chdir('/Users/camilledemers/Documents/03-corpus/2-sous-corpus/' + acteur + '/')
print('Pages (total) dans les sous-corpus : ' + str(liste_total) + ' (certaines pages peuvent avoir plus d\'un tag et donc apparaître deux fois).')
print('Pages (uniques) dans les sous-corpus : ' + str(len(liste_fichiers)))
print('\nIl y a donc {} pages de notre corpus global qui ne se retrouvent pas dans le sous-corpus'.format(str(len_corpus_global - len(liste_fichiers))))


Pages dans le corpus global : 4342
Pages (total) dans les sous-corpus : 6138 (certaines pages peuvent avoir plus d'un tag et donc apparaître deux fois).
Pages (uniques) dans les sous-corpus : 4141

Il y a donc 201 pages de notre corpus global qui ne se retrouvent pas dans le sous-corpus


À partir d'ici, on peut aller analyser les sous-corpus constitués en changeant le path et le 'acteur' dans le Notebook *Prétraitement.ipynb*  (on pourrait d'ailleurs changer le nom de la variable 'acteur' pour indiquer plutôt le critère/thème de regroupement de manière générale)